In [15]:
import numpy as np

from sb3_contrib import RecurrentPPO
from stable_baselines3.common.evaluation import evaluate_policy
from sb3_contrib.common.recurrent.type_aliases import RNNStates
import gym
import torch as th

Using cuda device
Creating environment from the given name 'CartPole-v1'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [131]:
from active_critic.utils.gym_utils import make_policy_dict

In [132]:
pd = make_policy_dict()

In [133]:
for key in pd:
    print(key)

assembly
basketball
binpicking
boxclose
buttonpresstopdown
buttonpresstopdownwall
buttonpress
buttonpresswall
coffeebutton
coffeepull
coffeepush
dialturn
disassemble
doorclose
doorlock
dooropen
doorunlock
handinsert
drawerclose
draweropen
faucetopen
faucetclose
hammer
handlepressside
handlepress
handlepullside
handlepull
leverpull
pickplacewall
pickoutofhole
reach
pushback
push
pickplace
plateslide
plateslideside
plateslideback
plateslidebackside
pegunplugside
soccer
stickpush
stickpull
pushwall
reachwall
shelfplace
sweepinto
sweep
windowopen
windowclose


In [67]:
env = gym.make("CartPole-v1")

In [82]:
model = RecurrentPPO("MlpLstmPolicy", "CartPole-v1", verbose=1)

Using cuda device
Creating environment from the given name 'CartPole-v1'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [83]:
lstm_stats = model._last_lstm_states

In [84]:
rnn_states_list = [RNNStates(pi=lstm_stats[0], vf=lstm_stats[1])]

In [85]:
obs =  th.tensor(env.reset(),device='cuda').reshape([1,-1])

In [127]:
actions, values, log_prob, rnn_states = model.policy.forward(obs = obs, lstm_states=rnn_states_list[0], episode_starts = th.ones([1], device='cuda'), deterministic=False)

In [128]:
ent_weight = 1e-2

eval actions:         
obs: th.Tensor,
actions: th.Tensor,
lstm_states: RNNStates,
episode_starts: th.Tensor,

-> 
values, log_prob, distribution.entropy()

In [129]:
_, log_prob, entropy = model.policy.evaluate_actions(obs, actions, rnn_states, episode_starts = th.ones([1], device='cuda'))
#print(f'max log: {log_prob.max()}')

prob_true_act = th.exp(log_prob).mean()
log_prob = log_prob.mean()
entropy = entropy.mean()


ent_loss = ent_weight * entropy
neglogp = -log_prob
loss = neglogp + ent_loss

In [130]:
loss

tensor(0.7000, device='cuda:0', grad_fn=<AddBackward0>)

In [22]:
model.policy.lstm_actor.forward(obs = env.reset(), lstm_states=lstm_stats, epsidode_starts = th.zeros([1]))

TypeError: LSTM.forward() got an unexpected keyword argument 'obs'

In [10]:
from imitation.algorithms.bc import BC
from active_critic.utils.gym_utils import (
    make_vec_env,
    make_dummy_vec_env,
    sample_expert_transitions_rollouts,
    make_pomdp_rollouts,
    make_dummy_vec_env_pomdp,
    get_avr_succ_rew_det
)

In [11]:
env_tag = 'pickaplce'
seq_len = 100
n_demonstrations

env, vec_expert = make_dummy_vec_env(name=env_tag, seq_len=seq_len)
val_env, _ = make_dummy_vec_env(name=env_tag, seq_len=seq_len)

transitions, rollouts = sample_expert_transitions_rollouts(
    vec_expert.predict, val_env, n_demonstrations)

NameError: name 'sample_expert_transitions_rollouts' is not defined